# Data Cleaning 4: Untagged PER Entities

Strategy 1: Exact Matches

Strategy 2: Fuzzy Matches

In [1]:
from IPython.display import clear_output
import spacy
from spacy import displacy

import pickle

In [2]:
import pandas as pd

In [3]:
file = open("Files_Cleaning3/NER_data_Cleaned3.p", 'rb')
docs = pickle.load(file)

# Exact Matching to Confirmed Names

Let's generate a list of PER entity strings:

In [6]:
peoplestrings =[]
for doc, context in docs:
    for ent in doc.ents:
        if ent.label_=='PER':
            peoplestrings.append(ent.text)         

In [7]:
len(peoplestrings)

32342

In [8]:
len(set(peoplestrings))

14263

In [9]:
peoplestrings = list(set(peoplestrings))

In [34]:
untaggedPERS = []

for string in peoplestrings: 
    strlist = string.split()
    for doc, context in docs:
        for token in doc:
            if token.ent_iob==2:
                start = token.i
                b=0
                while (doc[start+b].ent_iob==2) and (doc[start+b].text.lower() == strlist[b].lower()) and len(strlist)>=b:
                    b= b+1
                if b !=0:
                    string = str(string)
                    end = start + b
                    docid = context['id']
                    entry = [docid, start, end, string]
                    untaggedPERS.append(entry)

IndexError: list index out of range

In [36]:
untaggedPERS

[[230, 247, 248, 'Joseph Lopa'],
 [244, 228, 229, 'Joseph Lopa'],
 [266, 125, 126, 'Joseph Lopa'],
 [287, 190, 191, 'Joseph Lopa'],
 [310, 86, 87, 'Joseph Lopa'],
 [313, 135, 136, 'Joseph Lopa'],
 [325, 184, 185, 'Joseph Lopa'],
 [366, 331, 332, 'Joseph Lopa'],
 [1206, 1731, 1732, 'Joseph Lopa'],
 [1206, 2396, 2397, 'Joseph Lopa'],
 [1352, 50, 51, 'Joseph Lopa'],
 [1384, 102, 103, 'Joseph Lopa'],
 [1384, 127, 128, 'Joseph Lopa'],
 [1445, 88, 89, 'Joseph Lopa'],
 [1747, 141, 142, 'Joseph Lopa'],
 [1747, 171, 172, 'Joseph Lopa'],
 [1788, 57, 58, 'Joseph Lopa'],
 [2307, 572, 573, 'Joseph Lopa'],
 [2317, 14, 15, 'Joseph Lopa'],
 [2462, 137, 138, 'Joseph Lopa'],
 [2834, 59, 60, 'Joseph Lopa'],
 [2887, 138, 139, 'Joseph Lopa'],
 [2963, 47, 48, 'Joseph Lopa'],
 [2963, 75, 76, 'Joseph Lopa'],
 [3467, 433, 434, 'Joseph Lopa'],
 [4469, 1654, 1655, 'Joseph Lopa'],
 [4764, 123, 124, 'Joseph Lopa'],
 [4835, 322, 323, 'Joseph Lopa'],
 [4835, 857, 858, 'Joseph Lopa'],
 [6178, 137, 138, 'Joseph Lopa']